In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from datasets.dataset_dict import DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding
from sklearn import preprocessing

df = pd.read_csv('IMDB Dataset.csv')
label_encoder = preprocessing.LabelEncoder()
df['sentiment']= label_encoder.fit_transform(df['sentiment'])

dataset_dict = {
    'review': df['review'].tolist(),
    'sentiment': df['sentiment'].tolist()
}

imdb_dataset = Dataset.from_dict(dataset_dict)

train_test = imdb_dataset.train_test_split(test_size=0.2, seed=42)
train_test['rem_data'] = train_test['test']
del train_test['test']

test = train_test['rem_data'].train_test_split(test_size=0.5, seed=42)
test['validation'] = test['train']
del test['train']

imdb_data = DatasetDict({'train': train_test['train'], 'validation': test['validation'], 'test': test['test']})


# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["review"], padding = True, truncation=True)

tokenized_datasets = imdb_data.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets['train']



Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 40000
})

In [ ]:
len(df), len(imdb_dataset), len(train_test['train']), len(test['validation']), len(test['test'])

(50000, 50000, 40000, 5000, 5000)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["review"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
  print(batch)
  break
{k: v.shape for k, v in batch.items()}

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': tensor([1, 0, 0, 1, 1, 1, 1, 1]), 'input_ids': tensor([[  101,  1045,  2066,  ...,     0,     0,     0],
        [  101, 10433,  2210,  ...,     0,     0,     0],
        [  101,  2178,  4756,  ...,     0,     0,     0],
        ...,
        [  101,  2096,  2025,  ...,     0,     0,     0],
        [  101,  2383,  3728,  ...,     0,     0,     0],
        [  101,  7656, 18003,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from tqdm.auto import tqdm
import torch


model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))
loss_epoch = {}
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss_epoch[epoch] = loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/15000 [00:00<?, ?it/s]

In [ ]:
import numpy as np
losses_np = np.array([x.item() for x in loss_epoch.values()])
losses_np


array([0.03507322, 0.13126294, 0.23229547])

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

model.eval()

all_predictions = []
all_labels = []
val_loss={}
for epoch in range(num_epochs):
  for batch in eval_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      val_loss[epoch]=outputs.loss

      logits = outputs.logits
      # print(logits)
      predictions = torch.argmax(logits, dim=-1).cpu().numpy()
      labels = batch["labels"].cpu().numpy()

      all_predictions.extend(predictions)
      all_labels.extend(labels)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)

# Calculate precision, recall, F1-score, and support
precision, recall, f1_score, support = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

# Calculate confusion matrix
confusion = confusion_matrix(all_labels, all_predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")
print(f"Confusion Matrix:\n{confusion}")



Accuracy: 0.9476
Precision: 0.9425933360096347
Recall: 0.9517632752330766
F1 Score: 0.9471561113352158
Confusion Matrix:
[[7170  429]
 [ 357 7044]]


In [ ]:
import numpy as np
val_losses_np = np.array([x.item() for x in val_loss.values()])
val_losses_np


array([0.00246045, 0.00246045, 0.00246045])

Test Results

In [ ]:
# model.eval()
# for batch in test_dataloader:
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)

import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

model.eval()

all_predictions = []
all_labels = []
val_loss1={}
for epoch in range(num_epochs):
  for batch in test_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      val_loss1[epoch]=outputs.loss

      logits = outputs.logits
      # print(logits)
      predictions = torch.argmax(logits, dim=-1).cpu().numpy()
      labels = batch["labels"].cpu().numpy()

      all_predictions.extend(predictions)
      all_labels.extend(labels)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)

# Calculate precision, recall, F1-score, and support
precision, recall, f1_score, support = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

# Calculate confusion matrix
confusion = confusion_matrix(all_labels, all_predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")
print(f"Confusion Matrix:\n{confusion}")
print(val_loss1)

import numpy as np
val_losses_np1 = np.array([x.item() for x in val_loss1.values()])
val_losses_np1

Accuracy: 0.9362
Precision: 0.9303163796555867
Recall: 0.9412479740680713
F1 Score: 0.9357502517623363
Confusion Matrix:
[[7074  522]
 [ 435 6969]]
{0: tensor(0.0866, device='cuda:0'), 1: tensor(0.0866, device='cuda:0'), 2: tensor(0.0866, device='cuda:0')}


array([0.08658415, 0.08658415, 0.08658415])